In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/mitbih_database.zip'
extract_path = '/content/mitbih_database/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [3]:
extracted_path = '/content/mitbih_database/mitbih_database'
extracted_files = os.listdir(extracted_path)
print(extracted_files)

['222annotations.txt', '231.csv', '232.csv', '223.csv', '223annotations.txt', '124annotations.txt', '213annotations.txt', '201.csv', '202.csv', '228annotations.txt', '107annotations.txt', '219annotations.txt', 'mitbih_database', '220.csv', '123.csv', '121.csv', '118annotations.txt', '217.csv', '117.csv', '210annotations.txt', '108.csv', '215.csv', '117annotations.txt', '205.csv', '221annotations.txt', '215annotations.txt', '106.csv', '209.csv', '221.csv', '102annotations.txt', '111.csv', '123annotations.txt', '203.csv', '101annotations.txt', '205annotations.txt', '222.csv', '104.csv', '101.csv', '212annotations.txt', '112.csv', '233annotations.txt', '100annotations.txt', '212.csv', '111annotations.txt', '103.csv', '118.csv', '207.csv', '114annotations.txt', '106annotations.txt', '104annotations.txt', '231annotations.txt', '113.csv', '232annotations.txt', '119annotations.txt', '200annotations.txt', '208.csv', '219.csv', '228.csv', '105annotations.txt', '116.csv', '109annotations.txt', '

In [4]:
!pip install PyWavelets
!pip install --upgrade seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 76.0 MB/s eta 0:00:00


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import itertools
import collections
import pywt
from scipy import stats
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import keras
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential , Model
from keras.layers import Dense, LSTM, Dropout, Softmax, Bidirectional, Flatten
import math
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical, plot_model
from keras import regularizers

In [6]:
classes = ['N', 'A', 'V', 'F', 'f']
n_classes = 5
count_classes = [0]*5

X = list()
y = list()

In [7]:
path = '/content/mitbih_database/'
filenames = next(os.walk(extracted_path))[2]
records = list()
annotations = list()
filenames.sort()
print(filenames)

['100.csv', '100annotations.txt', '101.csv', '101annotations.txt', '102.csv', '102annotations.txt', '103.csv', '103annotations.txt', '104.csv', '104annotations.txt', '105.csv', '105annotations.txt', '106.csv', '106annotations.txt', '107.csv', '107annotations.txt', '108.csv', '108annotations.txt', '109.csv', '109annotations.txt', '111.csv', '111annotations.txt', '112.csv', '112annotations.txt', '113.csv', '113annotations.txt', '114.csv', '114annotations.txt', '115.csv', '115annotations.txt', '116.csv', '116annotations.txt', '117.csv', '117annotations.txt', '118.csv', '118annotations.txt', '119.csv', '119annotations.txt', '121.csv', '121annotations.txt', '122.csv', '122annotations.txt', '123.csv', '123annotations.txt', '124.csv', '124annotations.txt', '200.csv', '200annotations.txt', '201.csv', '201annotations.txt', '202.csv', '202annotations.txt', '203.csv', '203annotations.txt', '205.csv', '205annotations.txt', '207.csv', '207annotations.txt', '208.csv', '208annotations.txt', '209.csv'

In [8]:
for f in filenames:
    filename, file_extension = os.path.splitext(f)
    if(file_extension == '.csv'):
        records.append(path + "mitbih_database/" +filename + file_extension)
    else:
        annotations.append(path + "mitbih_database/" +filename + file_extension)

In [9]:
file_name = os.path.basename(annotations[0])
print(file_name)

100annotations.txt


In [10]:
def denoise(data):
    w = pywt.Wavelet('sym4')
    maxlev = pywt.dwt_max_level(len(data), w.dec_len)
    threshold = 0.04

    coeffs = pywt.wavedec(data, 'sym4', level=maxlev)
    for i in range(1, len(coeffs)):
        coeffs[i] = pywt.threshold(coeffs[i], threshold*max(coeffs[i]))

    datarec = pywt.waverec(coeffs, 'sym4')

    return datarec

In [11]:
def is_number(value):
    try:
        int(value)
        return True
    except ValueError:
        return False

In [12]:
import csv
import numpy as np
window_size = 180
for idx, record_file in enumerate(records):
    with open(record_file, 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        signal_data = [int(row[1]) for row in reader if is_number(row[1])]

    signal_data = denoise(signal_data)

    with open(annotations[idx], 'r') as filepostfix:
        annotation_lines = filepostfix.readlines()

    for line in annotation_lines[1:]:
        parts = list(filter(None, line.split(' ')))

        try:
            pos = int(parts[1])
            label = parts[2]

            if label in classes:
                class_idx = classes.index(label)
                count_classes[class_idx] += 1

                if window_size <= pos < len(signal_data) - window_size:
                    segment = signal_data[pos - window_size: pos + window_size]
                    X.append(segment)
                    y.append(class_idx)
        except ValueError:
            print("invalid annotation line")

print(np.shape(X), np.shape(y))


(86470, 360) (86470,)


In [13]:
X_df = pd.DataFrame(X)

In [14]:
y_df = pd.DataFrame(y)

In [15]:
class_dist = y_df.value_counts()
class_dist

,count
0,
0,75011
2,7129
1,2546
4,982
3,802


In [16]:
train_data = pd.concat([X_df, y_df.reset_index(drop=True)], axis=1)
train_data.columns = list(X_df.columns) + ['label']

balanced_train_data = []
for label in train_data['label'].unique():
    class_data = train_data[train_data['label'] == label]
    if len(class_data) > 5000:
        balanced_class_data = resample(class_data, replace=False, n_samples=5000, random_state=42)
    else:
        balanced_class_data = resample(class_data, replace=True, n_samples=5000, random_state=42)
    balanced_train_data.append(balanced_class_data)

balanced_train_data = pd.concat(balanced_train_data)

X_balanced = balanced_train_data.drop(columns=['label'])
y_balanced = balanced_train_data['label']

print(y_balanced.value_counts())

label
0    5000
1    5000
2    5000
4    5000
3    5000
Name: count, dtype: int64


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

In [18]:
X_test = pd.DataFrame(X_test)
X_train = pd.DataFrame(X_train)
y_test = pd.DataFrame(y_test)
y_train = pd.DataFrame(y_train)

In [19]:
X_test.shape

(5000, 360)

In [20]:
X_train_lstm = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lstm = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

y_train_lstm = to_categorical(y_train)
y_test_lstm = to_categorical(y_test)

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Flatten, Dropout, Dense, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

with tf.device('/GPU:0'):

    model1 = Sequential()
    model1.add(Bidirectional(LSTM(units=50, return_sequences=True, activation='tanh', input_shape=(X_train_lstm.shape[1], 1))))
    model1.add(Bidirectional(LSTM(units=10, return_sequences=True, activation='tanh')))
    model1.add(Flatten())
    model1.add(Dropout(0.5))
    model1.add(Dense(35))
    model1.add(Dense(5))
    model1.add(Softmax())

    model1.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

    checkpoint = ModelCheckpoint(
        filepath="best_model.keras",
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        verbose=1
    )

    history = model1.fit(
        X_train_lstm,
        y_train_lstm,
        batch_size=128,
        epochs=40,
        verbose=1,
        validation_data=(X_test_lstm, y_test_lstm),
        callbacks=[checkpoint]
    )

    score1 = model1.evaluate(X_test_lstm, y_test_lstm)
    print(f"Test Accuracy: {score1[1]}")


Num GPUs Available:  1
Epoch 1/40


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.2021 - loss: 2.1473
Epoch 1: val_accuracy improved from -inf to 0.20140, saving model to best_model.keras
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 63ms/step - accuracy: 0.2022 - loss: 2.1448 - val_accuracy: 0.2014 - val_loss: 1.6347
Epoch 2/40
156/157 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.2621 - loss: 1.5665
Epoch 2: val_accuracy improved from 0.20140 to 0.26180, saving model to best_model.keras
157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - accuracy: 0.2623 - loss: 1.5664 - val_accuracy: 0.2618 - val_loss: 1.6913
Epoch 3/40
156/157 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.3429 - loss: 1.5076
Epoch 3: val_accuracy improved from 0.26180 to 0.49300, saving model to best_model.keras
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 59ms/step - accuracy: 0.3434 - loss: 1.5068 - val_accuracy: 0.4930 - val_loss: 1.3141
Epoch 4/40
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5157 - loss: 1.2414
Epoch 4: val_accuracy improved from 0.49300 

In [22]:
from tensorflow.keras.models import load_model, Model
model1 = load_model('best_model.keras')

In [23]:
model_feat = Model(inputs=model1.layers[0].input, outputs=model1.get_layer('flatten_9').output)
feat_train = model_feat.predict(X_train_lstm)
print(feat_train.shape)
feat_test = model_feat.predict(X_test_lstm)
print(feat_test.shape)

625/625 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step
(20000, 7200)
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step
(5000, 7200)


In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

feat_train_scaled = scaler.fit_transform(feat_train)
feat_test_scaled = scaler.transform(feat_test)

print(feat_train_scaled.shape)
print(feat_test_scaled.shape)

(20000, 7200)
(5000, 7200)


In [42]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)

feat_train_pca = pca.fit_transform(feat_train_scaled)
feat_test_pca = pca.transform(feat_test_scaled)
print(feat_train_pca.shape)
print(feat_test_pca.shape)

(20000, 100)
(5000, 100)


In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(n_estimators=150, random_state=42)

rf_classifier.fit(feat_train_pca, np.argmax(y_train_lstm, axis=1))

y_pred = rf_classifier.predict(feat_test_pca)

accuracy = accuracy_score(np.argmax(y_test_lstm, axis=1), y_pred)
print(f"Hybrid Model Final Accuracy: {accuracy}")


Hybrid Model Final Accuracy: 0.9812


In [31]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_direct = RandomForestClassifier(n_estimators=150, random_state=42)
rf_direct.fit(X_train_scaled, y_train)

y_pred_direct = rf_direct.predict(X_test_scaled)

direct_rf_accuracy = accuracy_score(y_test, y_pred_direct)

print(f"Random Forest Model Accuracy: {direct_rf_accuracy}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Model Accuracy: 0.9798
